![Credit card being held in hand](credit_card.jpg)

Commercial banks receive _a lot_ of applications for credit cards. Many of them get rejected for many reasons, like high loan balances, low income levels, or too many inquiries on an individual's credit report, for example. Manually analyzing these applications is mundane, error-prone, and time-consuming (and time is money!). Luckily, this task can be automated with the power of machine learning and pretty much every commercial bank does so nowadays. In this workbook, you will build an automatic credit card approval predictor using machine learning techniques, just like real banks do.

### The Data

The data is a small subset of the Credit Card Approval dataset from the UCI Machine Learning Repository showing the credit card applications a bank receives. This dataset has been loaded as a `pandas` DataFrame called `cc_apps`. The last column in the dataset is the target value.

In [101]:
# Import necessary libraries
import pandas as pd
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report

# Load the dataset
cc_apps = pd.read_csv("cc_approvals.data", header=None) 
cc_apps.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13
0,b,30.83,0.000,u,g,w,v,1.25,t,t,1,g,0,+
1,a,58.67,4.460,u,g,q,h,3.04,t,t,6,g,560,+
2,a,24.50,0.500,u,g,q,h,1.50,t,f,0,g,824,+
3,b,27.83,1.540,u,g,w,v,3.75,t,t,5,g,3,+
4,b,20.17,5.625,u,g,w,v,1.71,t,f,0,s,0,+


In [102]:
# Identify missing values
print(cc_apps.isnull().sum())

0     0
1     0
2     0
3     0
4     0
5     0
6     0
7     0
8     0
9     0
10    0
11    0
12    0
13    0
dtype: int64


In [103]:
# Encode categorical variables
encoder = LabelEncoder()
for column in cc_apps.select_dtypes(include=['object']).columns:
    cc_apps[column] = encoder.fit_transform(cc_apps[column])



In [104]:
cc_apps.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13
0,2,156,0.000,2,1,13,8,1.25,1,1,1,0,0,0
1,1,328,4.460,2,1,11,4,3.04,1,1,6,0,560,0
2,1,89,0.500,2,1,11,4,1.50,1,0,0,0,824,0
3,2,125,1.540,2,1,13,8,3.75,1,1,5,0,3,0
4,2,43,5.625,2,1,13,8,1.71,1,0,0,2,0,0


In [105]:
# Separate features and target
X = cc_apps.iloc[:, :-1]  # All columns except the last
y = cc_apps.iloc[:, -1]   # The last column (target)

# Split data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [106]:
# Standardize the numerical features
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)


In [108]:
# Initialize and train Logistic Regression model
lr_model = LogisticRegression(random_state=42, max_iter=1000)
lr_model.fit(X_train, y_train)

# Evaluate Logistic Regression
lr_predictions = lr_model.predict(X_test)
lr_accuracy = accuracy_score(y_test, lr_predictions)
print(f"Logistic Regression Accuracy: {lr_accuracy}")
print("\nLogistic Regression Classification Report:")
print(classification_report(y_test, lr_predictions))


Logistic Regression Accuracy: 0.8405797101449275

Logistic Regression Classification Report:
              precision    recall  f1-score   support

           0       0.83      0.86      0.85        70
           1       0.85      0.82      0.84        68

    accuracy                           0.84       138
   macro avg       0.84      0.84      0.84       138
weighted avg       0.84      0.84      0.84       138



In [109]:
# Initialize and train Random Forest model
rf_model = RandomForestClassifier(random_state=42, n_estimators=100)
rf_model.fit(X_train, y_train)

# Evaluate Random Forest
rf_predictions = rf_model.predict(X_test)
rf_accuracy = accuracy_score(y_test, rf_predictions)
print(f"Random Forest Accuracy: {rf_accuracy}")
print("\nRandom Forest Classification Report:")
print(classification_report(y_test, rf_predictions))


Random Forest Accuracy: 0.8623188405797102

Random Forest Classification Report:
              precision    recall  f1-score   support

           0       0.88      0.84      0.86        70
           1       0.85      0.88      0.86        68

    accuracy                           0.86       138
   macro avg       0.86      0.86      0.86       138
weighted avg       0.86      0.86      0.86       138



In [110]:
print("Model Comparison:")
print(f"Logistic Regression Accuracy: {lr_accuracy}")
print(f"Random Forest Accuracy: {rf_accuracy}")

if lr_accuracy > rf_accuracy:
    print("Logistic Regression performed better.")
else:
    print("Random Forest performed better.")


Model Comparison:
Logistic Regression Accuracy: 0.8405797101449275
Random Forest Accuracy: 0.8623188405797102
Random Forest performed better.


In [111]:
# Grid search for Logistic Regression
lr_param_grid = {'C': [0.01, 0.1, 1, 10, 100]}
lr_grid_search = GridSearchCV(LogisticRegression(random_state=42, max_iter=1000), lr_param_grid, cv=5, scoring='accuracy')
lr_grid_search.fit(X_train, y_train)

# Best Logistic Regression
best_lr_model = lr_grid_search.best_estimator_
best_lr_accuracy = lr_grid_search.best_score_
print(f"Best Logistic Regression Accuracy: {best_lr_accuracy}")


Best Logistic Regression Accuracy: 0.8767731367731368


In [112]:
# Grid search for Random Forest
rf_param_grid = {
    'n_estimators': [50, 100, 150],
    'max_depth': [None, 10, 20],
    'min_samples_split': [2, 5, 10]
}
rf_grid_search = GridSearchCV(RandomForestClassifier(random_state=42), rf_param_grid, cv=5, scoring='accuracy')
rf_grid_search.fit(X_train, y_train)

# Best Random Forest
best_rf_model = rf_grid_search.best_estimator_
best_rf_accuracy = rf_grid_search.best_score_
print(f"Best Random Forest Accuracy: {best_rf_accuracy}")


Best Random Forest Accuracy: 0.8803767403767402


In [113]:
# Save the best model's accuracy
best_score = max(lr_accuracy, rf_accuracy, best_lr_accuracy, best_rf_accuracy)
print(f"Best Model's Accuracy: {best_score}")

Best Model's Accuracy: 0.8803767403767402
